In [1]:
!pip install pyod
!pip install dataheroes 
!pip install lightgbm
!pip install cubist
!pip install catboost

#
# !pip install preprocess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 4.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.1.1-py3-none-any.whl size=190078 sha256=2e40f908cd5c85fb9dba19427992a82dfa1b998348b5f18bc77a9e3e5a4356d8
  Stored in directory: /root/.cache/pip/wheels/a3/42/d7/48a53ffc1466bd63932f28583c64ebf442114db14a0bfa8c95
Successfully built pyod
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 46.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for licensing: filename=licensing-0.41-py3-none-any.whl size=13052 sha256=2346bcf652f6bebd18d2975a3bef4e7394f164b00121a1619b56086fa11872dc
  Stored in directory: /root/.cache/pip/wheels/f4/48/bb/b1b19bd137eab8995aa23393709fc223e9418ab21bba0836b1
Successfully built licensing
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.7.0
    Uninstalling importlib-metadata-6.7.0:
      Successfully unins

In [2]:
import cubist
from catboost import CatBoostRegressor
import numpy as np
import umap
from mlxtend.plotting.pca_correlation_graph import corr2_coeff
from pyod.models.ecod import ECOD
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.impute import SimpleImputer
from sklearn.linear_model import BayesianRidge
from sklearn.preprocessing import RobustScaler, MinMaxScaler,StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.feature_selection import f_regression, SelectKBest, chi2, VarianceThreshold
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.ensemble import ExtraTreesRegressor, AdaBoostRegressor, StackingRegressor, RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import RidgeCV, Lasso, ElasticNet, LinearRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR, SVR
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import KNNImputer
from sklearn.svm import LinearSVR, SVR
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from dataheroes import CoresetTreeServiceDTC




/opt/conda/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argume

In [3]:
"""Relevance Vector Machine classes for regression and classification."""
import numpy as np

from scipy.optimize import minimize
from scipy.special import expit

from sklearn.base import BaseEstimator, RegressorMixin, ClassifierMixin
from sklearn.metrics.pairwise import (
    linear_kernel,
    rbf_kernel,
    polynomial_kernel
)
from sklearn.multiclass import OneVsOneClassifier
from sklearn.utils.validation import check_X_y


class BaseRVM(BaseEstimator):

    """Base Relevance Vector Machine class.

    Implementation of Mike Tipping's Relevance Vector Machine using the
    scikit-learn API. Add a posterior over weights method and a predict
    in subclass to use for classification or regression.
    """

    def __init__(
        self,
        kernel='rbf',
        degree=3,
        coef1=None,
        coef0=0.0,
        n_iter=3000,
        tol=1e-3,
        alpha=1e-6,
        threshold_alpha=1e9,
        beta=1.e-6,
        beta_fixed=False,
        bias_used=True,
        verbose=False
    ):
        """Copy params to object properties, no validation."""
        self.kernel = kernel
        self.degree = degree
        self.coef1 = coef1
        self.coef0 = coef0
        self.n_iter = n_iter
        self.tol = tol
        self.alpha = alpha
        self.threshold_alpha = threshold_alpha
        self.beta = beta
        self.beta_fixed = beta_fixed
        self.bias_used = bias_used
        self.verbose = verbose

    def get_params(self, deep=True):
        """Return parameters as a dictionary."""
        params = {
            'kernel': self.kernel,
            'degree': self.degree,
            'coef1': self.coef1,
            'coef0': self.coef0,
            'n_iter': self.n_iter,
            'tol': self.tol,
            'alpha': self.alpha,
            'threshold_alpha': self.threshold_alpha,
            'beta': self.beta,
            'beta_fixed': self.beta_fixed,
            'bias_used': self.bias_used,
            'verbose': self.verbose
        }
        return params

    def set_params(self, **parameters):
        """Set parameters using kwargs."""
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def _apply_kernel(self, x, y):
        """Apply the selected kernel function to the data."""
        if self.kernel == 'linear':
            phi = linear_kernel(x, y)
        elif self.kernel == 'rbf':
            phi = rbf_kernel(x, y, self.coef1)
        elif self.kernel == 'poly':
            phi = polynomial_kernel(x, y, self.degree, self.coef1, self.coef0)
        elif callable(self.kernel):
            phi = self.kernel(x, y)
            if len(phi.shape) != 2:
                raise ValueError(
                    "Custom kernel function did not return 2D matrix"
                )
            if phi.shape[0] != x.shape[0]:
                raise ValueError(
                    "Custom kernel function did not return matrix with rows"
                    " equal to number of data points."""
                )
        else:
            raise ValueError("Kernel selection is invalid.")

        if self.bias_used:
            phi = np.append(phi, np.ones((phi.shape[0], 1)), axis=1)

        return phi

    def _prune(self):
        """Remove basis functions based on alpha values."""
        keep_alpha = self.alpha_ < self.threshold_alpha

        if not np.any(keep_alpha):
            keep_alpha[0] = True
            if self.bias_used:
                keep_alpha[-1] = True

        if self.bias_used:
            if not keep_alpha[-1]:
                self.bias_used = False
            self.relevance_ = self.relevance_[keep_alpha[:-1]]
        else:
            self.relevance_ = self.relevance_[keep_alpha]

        self.alpha_ = self.alpha_[keep_alpha]
        self.alpha_old = self.alpha_old[keep_alpha]
        self.gamma = self.gamma[keep_alpha]
        self.phi = self.phi[:, keep_alpha]
        self.sigma_ = self.sigma_[np.ix_(keep_alpha, keep_alpha)]
        self.m_ = self.m_[keep_alpha]

    def fit(self, X, y):
        """Fit the RVR to the training data."""
        X, y = check_X_y(X, y)

        n_samples, n_features = X.shape

        self.phi = self._apply_kernel(X, X)

        n_basis_functions = self.phi.shape[1]

        self.relevance_ = X
        self.y = y

        self.alpha_ = self.alpha * np.ones(n_basis_functions)
        self.beta_ = self.beta

        self.m_ = np.zeros(n_basis_functions)

        self.alpha_old = self.alpha_

        for i in range(self.n_iter):
            self._posterior()

            self.gamma = 1 - self.alpha_*np.diag(self.sigma_)
            self.alpha_ = self.gamma/(self.m_ ** 2)

            if not self.beta_fixed:
                self.beta_ = (n_samples - np.sum(self.gamma))/(
                    np.sum((y - np.dot(self.phi, self.m_)) ** 2))

            self._prune()

            if self.verbose:
                print("Iteration: {}".format(i))
                print("Alpha: {}".format(self.alpha_))
                print("Beta: {}".format(self.beta_))
                print("Gamma: {}".format(self.gamma))
                print("m: {}".format(self.m_))
                print("Relevance Vectors: {}".format(self.relevance_.shape[0]))
                print()

            delta = np.amax(np.absolute(self.alpha_ - self.alpha_old))

            if delta < self.tol and i > 1:
                break

            self.alpha_old = self.alpha_

        if self.bias_used:
            self.bias = self.m_[-1]
        else:
            self.bias = None

        return self


class RVR(BaseRVM, RegressorMixin):

    """Relevance Vector Machine Regression.

    Implementation of Mike Tipping's Relevance Vector Machine for regression
    using the scikit-learn API.
    """

    def _posterior(self):
        """Compute the posterior distriubtion over weights."""
        i_s = np.diag(self.alpha_) + self.beta_ * np.dot(self.phi.T, self.phi)
        self.sigma_ = np.linalg.inv(i_s)
        self.m_ = self.beta_ * np.dot(self.sigma_, np.dot(self.phi.T, self.y))

    def predict(self, X, eval_MSE=False):
        """Evaluate the RVR model at x."""
        phi = self._apply_kernel(X, self.relevance_)

        y = np.dot(phi, self.m_)

        if eval_MSE:
            MSE = (1/self.beta_) + np.dot(phi, np.dot(self.sigma_, phi.T))
            return y, MSE[:, 0]
        else:
            return y


class RVC(BaseRVM, ClassifierMixin):

    """Relevance Vector Machine Classification.

    Implementation of Mike Tipping's Relevance Vector Machine for
    classification using the scikit-learn API.
    """

    def __init__(self, n_iter_posterior=50, **kwargs):
        """Copy params to object properties, no validation."""
        self.n_iter_posterior = n_iter_posterior
        super(RVC, self).__init__(**kwargs)

    def get_params(self, deep=True):
        """Return parameters as a dictionary."""
        params = super(RVC, self).get_params(deep=deep)
        params['n_iter_posterior'] = self.n_iter_posterior
        return params

    def _classify(self, m, phi):
        return expit(np.dot(phi, m))

    def _log_posterior(self, m, alpha, phi, t):

        y = self._classify(m, phi)

        log_p = -1 * (np.sum(np.log(y[t == 1]), 0) +
                      np.sum(np.log(1-y[t == 0]), 0))
        log_p = log_p + 0.5*np.dot(m.T, np.dot(np.diag(alpha), m))

        jacobian = np.dot(np.diag(alpha), m) - np.dot(phi.T, (t-y))

        return log_p, jacobian

    def _hessian(self, m, alpha, phi, t):
        y = self._classify(m, phi)
        B = np.diag(y*(1-y))
        return np.diag(alpha) + np.dot(phi.T, np.dot(B, phi))

    def _posterior(self):
        result = minimize(
            fun=self._log_posterior,
            hess=self._hessian,
            x0=self.m_,
            args=(self.alpha_, self.phi, self.t),
            method='Newton-CG',
            jac=True,
            options={
                'maxiter': self.n_iter_posterior
            }
        )

        self.m_ = result.x
        self.sigma_ = np.linalg.inv(
            self._hessian(self.m_, self.alpha_, self.phi, self.t)
        )

    def fit(self, X, y):
        """Check target values and fit model."""
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)

        if n_classes < 2:
            raise ValueError("Need 2 or more classes.")
        elif n_classes == 2:
            self.t = np.zeros(y.shape)
            self.t[y == self.classes_[1]] = 1
            return super(RVC, self).fit(X, self.t)
        else:
            self.multi_ = None
            self.multi_ = OneVsOneClassifier(self)
            self.multi_.fit(X, y)
            return self

    def predict_proba(self, X):
        """Return an array of class probabilities."""
        phi = self._apply_kernel(X, self.relevance_)
        y = self._classify(self.m_, phi)
        return np.column_stack((1-y, y))

    def predict(self, X):
        """Return an array of classes for each input."""
        if len(self.classes_) == 2:
            y = self.predict_proba(X)
            res = np.empty(y.shape[0], dtype=self.classes_.dtype)
            res[y[:, 1] <= 0.5] = self.classes_[0]
            res[y[:, 1] >= 0.5] = self.classes_[1]
            return res
        else:
            return self.multi_.predict(X)

In [4]:
##################### PRE-PROCESSSS ##################


def preprocess(X_train: np.array, y_train: np.array, X_test: np.array, ids_train:np.array,ids_test:np.array, **kwargs):
    X_train, X_test = impute_mv(X_train, X_test,kwargs['impute_data'])
    print("Done with imputation")
#     x_mapped_to_isnan = np.isnan(X_train)
#     x_mapped_to_isinf = np.isinf(X_train)
#     for i in range(x_mapped_to_isnan.shape[0]):
#         for j in range(x_mapped_to_isnan.shape[1]):
#             if(x_mapped_to_isnan[i,j]==True):
#                 print(f'anomaly isnan at:: {i},{j}')
#     for i in range(x_mapped_to_isinf.shape[0]):
#         for j in range(x_mapped_to_isinf.shape[1]):
#             if(x_mapped_to_isinf[i,j]==True):
#                 print(f'anomaly isinf at:: {i},{j}')

#     print(x_mapped_to_isnan)
    
#     print(x_mapped_to_isinf)
#     print(X_test)
    # TODO

#     X_train, X_test = scale_data(X_train, X_test,'min_max')
#     print("Done with Scaling")
    X_train,y_train,ids_train = detect_remove_outliers(X_train,y_train, ids_train,kwargs['outlier_detection'],kwargs['contaminationRate'])
    print("Done with outliers")

    X_train, X_test = select_features(X_train, y_train, X_test,kwargs['feature_selection'],featuresNo=kwargs['featuresNo'])
    print("Done with Feature selection")
    
    X_train, X_test = scale_data(X_train, X_test,kwargs['scale_data'])
    print("Done with Scaling")
#     X_train, X_test = reduce_dim(X_train, X_test,kwargs['reduce_dim'])
#     print("Done with reducing dim")

#     X_train, X_test = scale_data(X_train, X_test,kwargs['scale_data'])
#     print("Done with Scaling")
    x_mapped_to_isnan = np.isnan(X_train)
    x_mapped_to_isinf = np.isinf(X_train)
    for i in range(x_mapped_to_isnan.shape[0]):
        for j in range(x_mapped_to_isnan.shape[1]):
            if(x_mapped_to_isnan[i,j]==True):
                print(f'anomaly isnan at:: {i},{j}')
    for i in range(x_mapped_to_isinf.shape[0]):
        for j in range(x_mapped_to_isinf.shape[1]):
            if(x_mapped_to_isinf[i,j]==True):
                print(f'anomaly isinf at:: {i},{j}')

    return X_train, y_train, X_test,ids_train,ids_test


def reduce_dim(X_train, X_test, method: str = 'PCA'):
    if method == 'PCA':
        reducer = PCA(n_components='mle', svd_solver='auto')

    elif method == 'UMAP':
        reducer = umap.UMAP()

    else:
        return X_train, X_test

    X_train = reducer.fit_transform(X_train)
    X_test = reducer.transform(X_test)
    return X_train, X_test


def select_features(X_train: np.array, y_train: np.array, X_test: np.array, method:str = 'correlation', featuresNo=175):
    if method == 'correlation':
        X_train, X_test = remove_correlated(X_train, X_test)
        # Select k best
        fs = SelectKBest(score_func=f_regression, k=featuresNo)
        X_train = fs.fit_transform(X_train, y_train)
        X_test = fs.transform(X_test)
    elif method=='kBest':
        fs = SelectKBest(score_func=f_regression, k=featuresNo)
        X_train = fs.fit_transform(X_train, y_train)
        X_test = fs.transform(X_test)
        
    else:
        raise Exception(f"Select_features: {method} is not implemented")
        
    # # Chi
    # f_p_values = chi2(X_train, y_train)
    # print(f_p_values)
    return X_train, X_test


def remove_correlated(X_train: np.array, X_test: np.array):
    # Constant features
    var_threshold = VarianceThreshold(threshold=0)  # threshold = 0 for constant
    var_threshold.fit_transform(X_train)
    var_threshold.transform(X_test)

    # Correlated
    cor = corr2_coeff(X_train.T, X_train.T)
    p = np.argwhere(np.triu(np.isclose(cor, 1), 1))
    X_train = np.delete(X_train, p[:, 1], axis=1)
    X_test = np.delete(X_test, p[:, 1], axis=1)
    return X_train, X_test


def scale_data(X_train: np.array, X_test: np.array, method: str = 'min_max'):
    if method == 'robust':
        transformer = RobustScaler()
    elif method == 'min_max':
        transformer = MinMaxScaler()
    elif method == 'NONE':
        return X_train, X_test
    elif method =='standard':
        transformer = StandardScaler()
    else:
        raise Exception(f"Scale: {method} is not implemented")

    X_train = transformer.fit_transform(X_train)
    X_test = transformer.transform(X_test)
    return X_train, X_test


def impute_mv(X_train: np.array, X_test: np.array, method: str = 'KNN'):
    if method == 'median':
        imp = SimpleImputer(missing_values=np.nan, strategy='median')
    elif method == 'mean':
        imp = SimpleImputer(missing_values=np.nan,strategy='mean')
    elif method == 'KNN':
        K=3
        imp = KNNImputer(n_neighbors=K)
    elif method == 'iterative': #aka mice
        imp = IterativeImputer(estimator=BayesianRidge(), n_nearest_features=None, imputation_order='ascending')

    else:
        raise Exception(f"Impute: {method} is not implemented")

    X_train = imp.fit_transform(X_train)
    X_test = imp.fit_transform(X_test)

    return X_train, X_test


def detect_remove_outliers(X_train: np.array, y_train: np.array, ids_train: np.array,method:str ='ECOD',contaminationRate=0.05):
    # TODO
    train_pred_indices = detect_outlier_obs(X_train, y_train, ids_train,method,contaminationRate)
    #TODO:: UPDATE TO GET THE SPECIFIC INDICES BACK
    X_train = (X_train[train_pred_indices])[0]
    y_train = (y_train[train_pred_indices])[0]
    ids_train = (ids_train[train_pred_indices])[0]
    print(X_train.shape)
    return X_train , y_train , ids_train


def detect_outlier_obs(X_train: np.array, y_train: np.array, X_test: np.array, method: str = 'ECOD',contaminationRate=0.05):
    train_pred, test_pred = [], []
    CONTAMINATION=contaminationRate
    if method == 'ECOD':
        ecod = ECOD(contamination=CONTAMINATION)
        ecod.fit(X_train)
        tmp_list = np.array(ecod.labels_).tolist()
#         get labels and then adjust to have indices to remove outliers
        s= len(tmp_list)
        ones_removed=0
        for i in range(s):
            if(tmp_list[i-ones_removed]==0):
                tmp_list[i-ones_removed]=i
            elif(tmp_list[i-ones_removed]==1):
                del tmp_list[i-ones_removed]
                ones_removed+=1
        train_pred.append(tmp_list)
                
    elif method == 'isolation_forest':
        clf = IsolationForest(n_estimators=150, max_samples='auto', contamination=CONTAMINATION)
        tmp_list = np.array(clf.fit_predict(X_train)).tolist()
#         get labels and then adjust to have indices to remove outliers
        s= len(tmp_list)
        ones_removed=0
        for i in range(s):
            if(tmp_list[i-ones_removed]==1):
                tmp_list[i-ones_removed]=i
            elif(tmp_list[i-ones_removed]==-1):
                del tmp_list[i-ones_removed]
                ones_removed+=1
        train_pred.append(tmp_list)

    elif method=="coresets":
        tree = CoresetTreeServiceDTC(optimized_for = 'cleaning')
        tree = tree.build(X=X_train,y=y_train, chunk_size=-1)
        result = tree.get_cleaning_samples(int(CONTAMINATION*X_train.shape[0]))
        tree.remove_samples(result['idx'])
        res = tree.get_cleaning_samples(X_train.shape[0])
        train_pred.append(res['idx'])

    else:
        raise Exception(f"Detect: {method} is not implemented")
    train_pred[0].sort() #in practice not needed but a neat thing to do 
    print(len(train_pred[0]))
    return train_pred

In [5]:
# test
# def read_data(X_train_path, y_train_path, X_test_path):
#     X_train = np.genfromtxt(X_train_path, delimiter=",")
#     y_train = np.genfromtxt(y_train_path, delimiter=",")
#     X_test = np.genfromtxt(X_test_path, delimiter=",")
#     return X_train, y_train, X_test

# methods={
#         "impute_data": ['median','mean','iterative','KNN'],
#         "outlier_detection":['ECOD','isolation_forest','coresets'],
#         "feature_selection":['correlation',],
#         "scale_data":['robust','min_max','NONE'],
#         "reduce_dim":['PCA','UMAP','NONE']
#     }

# for impute_method in methods['impute_data']:
#     for outlier_detection_method in methods['outlier_detection']:
#         for feature_selection_method in methods['feature_selection']:
#             for scale_data_method in methods['scale_data']:
#                 for reduce_dim_method in methods['reduce_dim']:
#                     print(f'impute:: {impute_method},outlier:: {outlier_detection_method}, feature_select:: {feature_selection_method}, scale_data:: {scale_data_method},reduce_dim:: {reduce_dim_method}')
#                     xTrainPath="/kaggle/input/aml-task1-allfiles/X_train.csv"
#                     yTrainPath="/kaggle/input/aml-task1-allfiles/y_train.csv"
#                     xTestPath="/kaggle/input/aml-task1-allfiles/X_test.csv"
#                     X_train, y_train, X_test = read_data(X_train_path=xTrainPath,
#                                                          y_train_path=yTrainPath,
#                                                          X_test_path=xTestPath)
#                     ids_train, ids_test = X_train[1:, 0], X_test[1:, 0].astype(int)
#                     X_train, y_train, X_test = X_train[1:, 1:], y_train[1:, 1:].ravel(), X_test[1:, 1:]
#                     X_train, y_train, X_test,ids_train,ids_test = preprocess(X_train, y_train, X_test,ids_train, ids_test,
#                             impute_data= impute_method,
#                             outlier_detection=outlier_detection_method,
#                             feature_selection=feature_selection_method,
#                             scale_data=scale_data_method,
#                             reduce_dim=reduce_dim_method)
                    


In [11]:
###################### Train ######################


np.random.seed(42)


def read_data(X_train_path, y_train_path, X_test_path):
    X_train = np.genfromtxt(X_train_path, delimiter=",")
    y_train = np.genfromtxt(y_train_path, delimiter=",")
    X_test = np.genfromtxt(X_test_path, delimiter=",")
    return X_train, y_train, X_test



def get_model(method: int = 3):
    
    if method == 0:
        estimators = [
            ('xgb', XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, colsample_bytree=0.8)),
            ('extratree', ExtraTreesRegressor(n_estimators=1000, random_state=0)),
#             ('adaboost', AdaBoostRegressor(n_estimators=1000, random_state=0)),
            ('lgbm', LGBMRegressor()),
#             ('svr_rbf', SVR(kernel='rbf')),
#             ('mn', KNeighborsRegressor()),
            ('rvm', RVR(alpha=1e-06)),
            ('cat', CatBoostRegressor()),
            ('gp', GaussianProcessRegressor(kernel=RationalQuadratic(alpha=0.5), random_state=42)),
#             ('cubist', cubist.Cubist()),
        ]
        model = StackingRegressor(estimators=estimators,
                                  final_estimator=RidgeCV())
    elif method == 1:
        estimators = [
            # ('lr', RidgeCV()),
            # ('lasso', Lasso(alpha=0.134694)),
            # ('enet', ElasticNet(alpha=0.201, l1_ratio=0.005)),
            # ('lm', LinearRegression()),
            # ('kernel_ridge', KernelRidge(alpha=2.0, kernel='polynomial', degree=1, coef0=0.005)),
            ('xgb', XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, colsample_bytree=0.8)),
            ('extratree', ExtraTreesRegressor(n_estimators=1000, random_state=0)),
            ('adaboost', AdaBoostRegressor(n_estimators=1000, random_state=0)),
            # ('svr_lin', SVR(kernel='linear')),
            # ['svr_rbf', SVR(kernel='rbf')],
            ('mn', KNeighborsRegressor()),
        ]
        model = StackingRegressor(estimators=estimators, final_estimator=RandomForestRegressor(n_estimators=100, random_state=42))
#TODO:: fix method2 error

    elif method == 2:
        estimators = [
            ('xgb', XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, colsample_bytree=0.8)),
            ('extratree', ExtraTreesRegressor(n_estimators=1000, random_state=0)),
            ('adaboost', AdaBoostRegressor(n_estimators=1000, random_state=0)),
            ('lgbm', LGBMRegressor()),
            ('svr_rbf', SVR(kernel='rbf')),
            ('mn', KNeighborsRegressor()),
            ('rvm', RVR(alpha=1e-06)),
            ('cat', CatBoostRegressor()),
            ('gp', GaussianProcessRegressor(kernel=RationalQuadratic(alpha=0.5), random_state=42)),
#             ('cubist', cubist.Cubist()),
        ]
        model = StackingRegressor(estimators=estimators,
                                  final_estimator=RidgeCV())
    elif method==3:
        model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, colsample_bytree=0.8)
    elif method==4:
        model = ExtraTreesRegressor(n_estimators=1000, random_state=0)
    elif method==5:
        model = AdaBoostRegressor(n_estimators=1000, random_state=0)
    elif method==6:
        model = LGBMRegressor()
    elif method==7:
        model = SVR(kernel='rbf')
    elif method==8:
        model = KNeighborsRegressor()
    elif method==9:
        model = RVR(alpha=1e-06)
    elif method==10:
        model = CatBoostRegressor()
    elif method==11:
        model =  GaussianProcessRegressor(kernel=RationalQuadratic(alpha=0.5), random_state=42)
    elif method==12:
        model = cubist.Cubist()
    else:
        raise Exception(f"Model: {method} is not implemented.")

    return model

def get_splits(X_train: np.array, nfolds: int = 10):
    kf = KFold(n_splits=nfolds, random_state=42, shuffle=True)
    return kf.split(X_train)
# impute:: median, outlier:: isolation_forest, feature_select:: correlation, scale_data:: NONE,reduce_dim:: PCA


def main():
#     methods={
#         "impute_data": ['iterative','median','mean','KNN'],
#         "outlier_detection":['coresets','ECOD','isolation_forest'],
#         "feature_selection":['correlation'],
#         "featuresNo":[100,150,200,250,300,350,400],
#         "scale_data":['robust','min_max','standard','NONE'],
#         "reduce_dim":['NONE','PCA','UMAP']
#     }
    
    methods={
        "impute_data": ['median'],
        "outlier_detection":['isolation_forest'],
#         "contamination_rate":[0.01,0.02,0.05,0.07,0.1,0.15,0.2],
        "contamination_rate":[0.05],
        "feature_selection":['kBest'],
#         "featuresNo":[100,150,200,250,300,350,400],
        "featuresNo":[200],
        "scale_data":['standard'],
        "reduce_dim":['NONE']
    }
    model_version = 0
    only_preprocess = False
    for impute_method in methods['impute_data']:
        for outlier_detection_method in methods['outlier_detection']:
            for feature_selection_method in methods['feature_selection']:
                for featureNo in methods['featuresNo']:
                    for contaminationRate in methods['contamination_rate']:
                        for scale_data_method in methods['scale_data']:
                            for reduce_dim_method in methods['reduce_dim']:
                                print(f'impute:: {impute_method}, outlier:: {outlier_detection_method}_{contaminationRate}, feature_select:: {feature_selection_method}_{featureNo}, scale_data:: {scale_data_method}, reduce_dim:: {reduce_dim_method}')                        
                                if scale_data_method == 'NONE'and reduce_dim_method == 'PCA':
                                    print("Skipping the combination of scale_data method:: NONE, reduce_dim_method:: PCA due to joint error")
                                    continue
                                xTrainPath="/kaggle/input/aml-task1-allfiles/X_train.csv"
                                yTrainPath="/kaggle/input/aml-task1-allfiles/y_train.csv"
                                xTestPath="/kaggle/input/aml-task1-allfiles/X_test.csv"
                                X_train, y_train, X_test = read_data(X_train_path=xTrainPath,
                                                                     y_train_path=yTrainPath,
                                                                     X_test_path=xTestPath)
                                ids_train, ids_test = X_train[1:, 0], X_test[1:, 0].astype(int)
                                X_train, y_train, X_test = X_train[1:, 1:], y_train[1:, 1:].ravel(), X_test[1:, 1:]
                                X_train, y_train, X_test,ids_train,ids_test = preprocess(X_train, y_train, X_test,ids_train, ids_test,
                                    impute_data= impute_method,
                                    outlier_detection=outlier_detection_method,
                                    contaminationRate=contaminationRate,
                                    feature_selection=feature_selection_method,
                                    featuresNo=featureNo,
                                    scale_data=scale_data_method,
                                    reduce_dim=reduce_dim_method)
                                print("Preprocessed.")
                                if only_preprocess:
                                    path_out_X_train_preprocessed='/kaggle/working/'+'X_train_preprocessed_'+impute_method+'_'+outlier_detection_method+'_'+str(contaminationRate)+'_'+feature_selection_method+'_'+str(featureNo)+'_'+scale_data_method+'_'+reduce_dim_method+'.csv'
                                    path_out_X_test_preprocessed='/kaggle/working/'+'X_test_preprocessed_'+impute_method+'_'+outlier_detection_method+'_'+str(contaminationRate)+'_'+feature_selection_method+'_'+str(featureNo)+'_'+scale_data_method+'_'+reduce_dim_method+'.csv'
                                    path_out_y_train_preprocessed='/kaggle/working/'+'y_train_preprocessed_'+impute_method+'_'+outlier_detection_method+'_'+str(contaminationRate)+'_'+feature_selection_method+'_'+str(featureNo)+'_'+scale_data_method+'_'+reduce_dim_method+'.csv'
                                    np.savetxt(path_out_X_train_preprocessed, X_train, delimiter=",", comments='')
                                    np.savetxt(path_out_X_test_preprocessed, X_test,  delimiter=",",  comments='')
                                    np.savetxt(path_out_y_train_preprocessed, y_train,  delimiter=",", header="id,y", comments='')
                                else:
                                    model = get_model(model_version)

                                    nfolds = 10
                                    splits = get_splits(X_train, nfolds)

                                    print("\nModels and folds.")

                                    r2 = 0
                                    for i, (train_index, test_index) in enumerate(splits):
                                        model.fit(X_train[train_index], y_train[train_index])
                                        pred = model.predict(X_train[test_index])
                                        score = r2_score(y_train[test_index], pred)
                                        r2 += score

                                        print(f"Fold {i} R2 score: {score}")

                                    print(f"\nAvg R2: {r2 / nfolds}")

                                    print("\nTrained.")

                                    model.fit(X_train, y_train)
                                    pred = model.predict(X_test)
                                    res = np.column_stack((ids_test, pred))
                                    path_out='/kaggle/working/'+'out_'+impute_method+'_'+outlier_detection_method+'_'+str(contaminationRate)+'_'+feature_selection_method+'_'+str(featureNo)+'_'+scale_data_method+'_'+reduce_dim_method+'.csv'
                                    np.savetxt(path_out, res, fmt=['%1i', '%1.4f'], delimiter=",", header="id,y", comments='')

                            #     X_train, y_train, X_test = preprocess(X_train, y_train, X_test)
    #                             try:
    #                                 X_train, y_train, X_test,ids_train,ids_test = preprocess(X_train, y_train, X_test,ids_train, ids_test,
    #                                     impute_data= impute_method,
    #                                     outlier_detection=outlier_detection_method,
    #                                     feature_selection=feature_selection_method,
    #                                     featuresNo=featureNo,
    #                                     scale_data=scale_data_method,
    #                                     reduce_dim=reduce_dim_method)
    #                                 print("Preprocessed.")
    #                                 if only_preprocess:
    #                                     path_out_X_train_preprocessed='/kaggle/working/'+'X_train_preprocessed_'+impute_method+'_'+outlier_detection_method+'_'+feature_selection_method+'_'+str(featureNo)+'_'+scale_data_method+'_'+reduce_dim_method+'.csv'
    #                                     path_out_X_test_preprocessed='/kaggle/working/'+'X_test_preprocessed_'+impute_method+'_'+outlier_detection_method+'_'+feature_selection_method+'_'+str(featureNo)+'_'+scale_data_method+'_'+reduce_dim_method+'.csv'
    #                                     path_out_y_train_preprocessed='/kaggle/working/'+'y_train_preprocessed_'+impute_method+'_'+outlier_detection_method+'_'+feature_selection_method+'_'+str(featureNo)+'_'+scale_data_method+'_'+reduce_dim_method+'.csv'
    #                                     np.savetxt(path_out_X_train_preprocessed, X_train, delimiter=",", comments='')
    #                                     np.savetxt(path_out_X_test_preprocessed, X_test,  delimiter=",",  comments='')
    #                                     np.savetxt(path_out_y_train_preprocessed, y_train,  delimiter=",", header="id,y", comments='')
    #                                 else:
    #                                     model = get_model()

    #                                     nfolds = 10
    #                                     splits = get_splits(X_train, nfolds)

    #                                     print("\nModels and folds.")

    #                                     r2 = 0
    #                                     for i, (train_index, test_index) in enumerate(splits):
    #                                         model.fit(X_train[train_index], y_train[train_index])
    #                                         pred = model.predict(X_train[test_index])
    #                                         score = r2_score(y_train[test_index], pred)
    #                                         r2 += score

    #                                         print(f"Fold {i} R2 score: {score}")

    #                                     print(f"\nAvg R2: {r2 / nfolds}")

    #                                     print("\nTrained.")

    #                                     model.fit(X_train, y_train)
    #                                     pred = model.predict(X_test)
    #                                     res = np.column_stack((ids_test, pred))
    #                                     path_out='/kaggle/working/'+'out_'+impute_method+'_'+outlier_detection_method+'_'+feature_selection_method+'_'+str(featureNo)+'_'+scale_data_method+'_'+reduce_dim_method+'.csv'
    #                                     np.savetxt(path_out, res, fmt=['%1i', '%1.4f'], delimiter=",", header="id,y", comments='')

    #                             except ValueError:
    #                                 print(f'Error in::impute:: {impute_method}, outlier:: {outlier_detection_method}, feature_select:: {feature_selection_method}, scale_data:: {scale_data_method}, reduce_dim:: {reduce_dim_method}')                                       
    #                                 pass


if __name__ == "__main__":
    main()

impute:: median, outlier:: isolation_forest_0.05, feature_select:: kBest_200, scale_data:: standard, reduce_dim:: NONE
Done with imputation
1151
(1151, 832)
Done with outliers
Done with Feature selection
Done with Scaling
Preprocessed.

Models and folds.
Fold 0 R2 score: 0.5361839764316751
Fold 1 R2 score: 0.5500909635579436
Fold 2 R2 score: 0.5562975624410124
Fold 3 R2 score: 0.4666050768988863
Fold 4 R2 score: 0.40604863590038753
Fold 5 R2 score: 0.6651928456446853
Fold 6 R2 score: 0.6389019791951429
Fold 7 R2 score: 0.4452669957933376
Fold 8 R2 score: 0.5705946246504346
Fold 9 R2 score: 0.5608811687756635

Avg R2: 0.5396063829289168

Trained.
